In [1]:
import tvm
from tvm import relay

import numpy as np

from tvm.contrib.download import download_testdata

# PyTorch imports
import torch
import torchvision

In [2]:
model_name = "resnet18"
model = getattr(torchvision.models, model_name)(pretrained=True)
model = model.eval()

# We grab the TorchScripted model via tracing
input_shape = [1, 3, 224, 224]
input_data = torch.randn(input_shape)
scripted_model = torch.jit.trace(model, input_data).eval()

/home/wendell/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/wendell/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/wendell/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [5]:
from PIL import Image

img_url = "https://github.com/dmlc/mxnet.js/blob/main/data/cat.png?raw=true"
img_path = download_testdata(img_url, "cat.png", module="data")
img = Image.open(img_path).resize((224, 224))

# Preprocess the image and convert to tensor
from torchvision import transforms

my_preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
img = my_preprocess(img)
img = np.expand_dims(img, 0)

In [6]:
input_name = "input0"
shape_list = [(input_name, img.shape)]
mod, params = relay.frontend.from_pytorch(scripted_model, shape_list)

In [18]:
from tvm.relay.expr_functor import ExprMutator

class ScheduleConv2d(ExprMutator):
    def __init__(self):
        super().__init__()

    def visit_call(self, expr):
        visit = super().visit_call(expr)
        print(expr.op)
        if expr.op != tvm.relay.op.get("nn.dropout"):
            return relay.annotation.on_device(visit, tvm.cuda(0))
        else:
            return visit

In [17]:
for gv in mod.get_global_vars():
    mod[gv] = ScheduleConv2d().visit(mod[gv])
mod = relay.transform.InferType()(mod)

nn.conv2d
on_device
on_device
on_device
nn.batch_norm
on_device
on_device
on_device
nn.relu
on_device
on_device
on_device
nn.max_pool2d
on_device
on_device
on_device
nn.conv2d
on_device
on_device
on_device
nn.batch_norm
on_device
on_device
on_device
nn.relu
on_device
on_device
on_device
nn.conv2d
on_device
on_device
on_device
nn.batch_norm
on_device
on_device
on_device
add
on_device
on_device
on_device
nn.relu
on_device
on_device
on_device
nn.conv2d
on_device
on_device
on_device
nn.batch_norm
on_device
on_device
on_device
nn.relu
on_device
on_device
on_device
nn.conv2d
on_device
on_device
on_device
nn.batch_norm
on_device
on_device
on_device
add
on_device
on_device
on_device
nn.relu
on_device
on_device
on_device
nn.conv2d
on_device
on_device
on_device
nn.batch_norm
on_device
on_device
on_device
nn.relu
on_device
on_device
on_device
nn.conv2d
on_device
on_device
on_device
nn.batch_norm
on_device
on_device
on_device
nn.conv2d
on_device
on_device
on_device
nn.batch_norm
on_device
on_devic